<a href="https://colab.research.google.com/github/morekajal/Large_Language_Models_with_PyTorch/blob/main/07_getbatch_logits_lossreporting_train_evalmode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

cuda


In [ ]:
with open('/content/drive/MyDrive/MyLLMWork/data/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [ ]:
string_to_int = {ch:i for i, ch in enumerate(chars) }
int_to_string = {i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [ ]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  # print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  # convert to device()
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs:')
print('x shape : ', x.shape)
print(x)
print('targets :')
print(y)

inputs:
x shape :  torch.Size([4, 8])
tensor([[59, 54, 65, 68, 58, 72,  9,  3],
        [73, 68,  1, 71, 74, 65, 58,  1],
        [65,  1, 76, 61, 68,  1, 76, 58],
        [54, 73,  1, 55, 58, 54, 72, 73]], device='cuda:0')
targets :
tensor([[54, 65, 68, 58, 72,  9,  3,  1],
        [68,  1, 71, 74, 65, 58,  1, 74],
        [ 1, 76, 61, 68,  1, 76, 58, 71],
        [73,  1, 55, 58, 54, 72, 73, 65]], device='cuda:0')


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is ', context, 'target is ', target)

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)


#### Lets try to build a Neural Net

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
a = torch.rand(2, 3, 5)
x , y , z = a.shape
print('x y z : ', x, y, z)
a = a.view(x, y, z)
print(a)
print(a.shape)

x y z :  2 3 5
tensor([[[0.0674, 0.8292, 0.6876, 0.3155, 0.2426],
         [0.4059, 0.7895, 0.7963, 0.5220, 0.4099],
         [0.6250, 0.6126, 0.2821, 0.1820, 0.2131]],

        [[0.7275, 0.1701, 0.0723, 0.4019, 0.9646],
         [0.1296, 0.3204, 0.1198, 0.9052, 0.9703],
         [0.0551, 0.6277, 0.0733, 0.5821, 0.3524]]])
torch.Size([2, 3, 5])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):  # Make 'targets' optional with a default value of None
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B * T, C)
      targets = targets.view(B * T)         # why we need to change the shape??
      # what does pytorch expect ->
      loss = F.cross_entropy(logits, targets)   #as target=None given inside forward, else part does not execute here

    return logits, loss

  def generate(self, index, max_new_tokens):
    #index is (B, T) array of indices in current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the last time step
      logits = logits[:, -1, :]             #(B, C)
      # apply softmax to get probabilitie
      probs = F.softmax(logits, dim=-1)     #(B, C)
      #sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim=1)
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

#torch.long -> int64 -> 64bytes
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


D]!10a5KPSVP66AA:EPY
b]uXtDIjEecwqMvR]*;*eg.1P]"Lp﻿k
B 4﻿zRLEcaKfdF8Zyj1D*X3)H.an﻿Ycn]gx*g-oo&gd
p10x[!Sk;"M8 y73 yXr(S;TSVq.FOE9cKfeDj27"m:-
fEH_Clw?iW:X(H_ZwL!WxArA*]u"viwc*0bUp
xbhXUXKHhN!Prr*0x[G3:w&m9108n!U,Ry&akxGNMfEtJ([uR4lKlrcC3n8C31cMh﻿3&a?wu83W *gG
woDIy7.v]HSbPY0clB.XCyp_oj?z5N5Kqzlf,4DHA5C:l8;MRkh9Ad_Hh1F:"J5﻿vH)j6IjowMbCQrb"Xr]kf
g-0,&ICayR2h;C
g-
pneEPAkNhhiaV*(LnH5Sx[D;T&]Uz[4w
FG
WE﻿gexUtw]ss
kdS[vo('mkV0'q9(u﻿MlA,RYmyrx5M
zZ3 6(7y'Nzdrv8f!q26JUX!E(z)TaeQcis_Sk"hT-byu!)0iDA2qtpB


Create Pytorch optimizer

In [ ]:
max_iters = 10000
#eval_interval = 2500
learning_rate = 3e-4

In [ ]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# for iter in range(max_iters):
#   #sample a batch of data
#   xb, yb = get_batch('train')

#   #evaluate the loss
#   logits, loss = model.forward(xb, yb)
#   optimizer.zero_grad(set_to_none=True)    #gradient set to none -> occupies smaller space
#   loss.backward()
#   optimizer.step()
# print(loss.item())

In [ ]:
# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

Still we get garbage text/token for bigram model

need to familiarize - just what the optimizer does for us and some of the differences/similarities between them*

1. Mean Squared Error (MSE): MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. Gradient Descent (GD): is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. Momentum: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. RMSprop: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. Adam: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. AdamW: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will be training.

To know, how well our model is generalizing

In [ ]:
eval_iters = 250
dropout = 0.2      #dropout randomly the neural networks

In [ ]:
@torch.no_grad()     #to make sure, pytorch doesnt use gradients at all here
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_iters == 0:       #modulus operator if remainder == 0
    losses = estimate_loss()
    print(f"step : {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

  xb, yb = get_batch('train')

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

step : 0, train loss: 3.2274, val loss: 3.2371
step : 250, train loss: 3.1707, val loss: 3.1987
step : 500, train loss: 3.1465, val loss: 3.1940
step : 750, train loss: 3.1497, val loss: 3.1632
step : 1000, train loss: 3.1129, val loss: 3.1364
step : 1250, train loss: 3.0836, val loss: 3.1037
step : 1500, train loss: 3.0682, val loss: 3.0873
step : 1750, train loss: 3.0512, val loss: 3.0589
step : 2000, train loss: 3.0138, val loss: 3.0355
step : 2250, train loss: 3.0155, val loss: 3.0474
step : 2500, train loss: 2.9770, val loss: 3.0136
step : 2750, train loss: 2.9633, val loss: 3.0034
step : 3000, train loss: 2.9601, val loss: 2.9558
step : 3250, train loss: 2.9523, val loss: 2.9583
step : 3500, train loss: 2.9244, val loss: 2.9346
step : 3750, train loss: 2.9140, val loss: 2.9384
step : 4000, train loss: 2.8887, val loss: 2.9199
step : 4250, train loss: 2.8561, val loss: 2.8993
step : 4500, train loss: 2.8414, val loss: 2.9013
step : 4750, train loss: 2.8542, val loss: 2.8567
step :